# make a new function to make a 2D slice based on 1D Earth model (or 2D or 3D)

In [1]:
# Nobuaki Fuji Oct 2025

# breaking news! confirmed by Lorette and Stéphanie, the CMB is ellipsoidal! 
# when working with geodetic earth models, r_PREM should be corrected as a function of latitude!

using Pkg


cd(@__DIR__)
Pkg.activate("../..")
ParamFile = "../test/testparam.csv"
include("../src/batchRevise.jl") 

myInclude("../src/DSM1D.jl")
using .DSM1D

  Activating project at `~/Documents/Github/flexibleDSM`
┌ Info: Including with Revise: ../src/DSM1D.jl
└ @ Main /Users/nobuaki/Documents/Github/flexibleDSM/OPTmotors/src/batchRevise.jl:25


  0.004688 seconds (2.17 k allocations: 108.156 KiB, 84.99% compilation time)


In [9]:
using Geodesy, Interpolations, StaticArrays, GMT, LinearAlgebra

In [3]:
# Example 1-D model (toy). Replace with your actual model arrays.
# r_model in meters from Earth center (0..R_ref), assume sorted ascending
R_ref = 6371e3                     # model reference radius (m)
r_model = collect(0.0:1000.0:R_ref)  # example radius grid
v_model = 5000 .- 0.5 .* (R_ref .- r_model) ./ 1000  # dummy profile

6372-element Vector{Float64}:
 1814.5
 1815.0
 1815.5
 1816.0
 1816.5
 1817.0
 1817.5
 1818.0
 1818.5
 1819.0
    ⋮
 4996.0
 4996.5
 4997.0
 4997.5
 4998.0
 4998.5
 4999.0
 4999.5
 5000.0

In [4]:
# create an interpolator in radius space (ensure interpolation domain covers used r)
itp = LinearInterpolation(r_model, v_model; extrapolation_bc=Flat())

6372-element extrapolate(interpolate((::Vector{Float64},), ::Vector{Float64}, Gridded(Linear())), Flat()) with element type Float64:
 1814.5
 1815.0
 1815.5
 1816.0
 1816.5
 1817.0
 1817.5
 1818.0
 1818.5
 1819.0
    ⋮
 4996.0
 4996.5
 4997.0
 4997.5
 4998.0
 4998.5
 4999.0
 4999.5
 5000.0

In [25]:
struct GeoPoint
    lat::Float64 # in degree
    lon::Float64 # in degree
    alt::Float64 # in metre
    ecef::SVector{3,Float64}
    radius::Float64 # in metre
end

function GeoPoint(lat, lon; alt=0.0)
    lla = LLA(deg2rad(lat), deg2rad(lon), alt)
    ecef_coords = ECEF(lla,wgs84)
    radius = norm([ecef_coords.x,ecef_coords.y,ecef_coords.z])
    GeoPoint(lat, lon, alt, SVector(ecef_coords.x, ecef_coords.y, ecef_coords.z),radius)
end



function topographyFromGeoPoint(p::GeoPoint; dlon=0.01, dlat=0.01)
    region = [p.lon - dlon, p.lon + dlon, p.lat - dlat, p.lat + dlat]
    topo = GMT.grdcut("@earth_relief_01m", region=region)
    return GMT.grd2xyz(topo).z[1]  # or mean(topo.z) if you prefer
end

topographyFromGeoPoint (generic function with 1 method)

In [32]:
p1 = GeoPoint(48.8566,2.3522) # Paris
LLAfromECEF(ECEF(collect(p1.ecef)...,wgs84))

DimensionMismatch: DimensionMismatch: No precise constructor for ECEF found. Length of input was 4.

In [21]:
p1 = GeoPoint(48.8566,2.3522) # Paris
p2 = GeoPoint(42.8,1.5) # Tarascon (à peu près)
topographyFromGeoPoint(p1)

grdblend [NOTICE]: Remote data courtesy of GMT data server oceania [http://oceania.generic-mapping-tools.org]

grdblend [NOTICE]: SRTM15 Earth Relief v2.7 at 01x01 arc minutes reduced by Gaussian Cartesian filtering (5.2 km fullwidth) [Tozer et al., 2019].
grdblend [NOTICE]:   -> Download 30x30 degree grid tile (earth_relief_01m_g): N30E000


ErrorException: type GMTdataset has no field z